In [ ]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

from core.data_sources import CLOBDataSource
from core.services.timescale_client import TimescaleClient
from decimal import Decimal


clob = CLOBDataSource()
ts_client = TimescaleClient(
    host=os.getenv("TIMESCALE_HOST", "63.250.52.93"),
    port=os.getenv("TIMESCALE_PORT", 5432),
    user=os.getenv("TIMESCALE_USER", "admin"),
    password=os.getenv("TIMESCALE_PASSWORD", "admin"),
    database="timescaledb"
)
await ts_client.connect()

In [ ]:
CONNECTOR_NAME = "binance_perpetual"
# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET) \
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE)) \
    .get_all_trading_pairs()
trading_pairs_available = await ts_client.get_available_pairs()
trading_pairs_available = [pair[1] for pair in trading_pairs_available if pair[0] == CONNECTOR_NAME]

trading_pairs = [trading_pair for trading_pair in trading_pairs_available if trading_pair in trading_pairs]

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer


resolution = "1s"
optimizer = StrategyOptimizer(engine="postgres",
                              root_path=root_path,
                              resolution=resolution,
                              db_client=ts_client,
                              db_host="63.250.52.93",
                              db_port=5433,
                              db_user="admin",
                              db_pass="admin",
                              )


In [ ]:
optimizer.get_all_study_names()

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
from hummingbot.strategy_v2.backtesting import DirectionalTradingBacktesting
#from research_notebooks.macd_mt.madbb_dynamic_config_generation import MACDMTConfigGenerator
from research_notebooks.macd_mt.madbb_fixed_pct_config_generation import MACDMTConfigGenerator

import datetime
import logging



logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

BACKTESTING_DAYS = 7

start_date = datetime.datetime.now() - datetime.timedelta(days=BACKTESTING_DAYS)
end_date = datetime.datetime.now()
logger.info("Generating config generator")

await optimizer._db_client.connect()
for trading_pair in trading_pairs:
    config_generator = MACDMTConfigGenerator(start_date=start_date, end_date=end_date, backtester=DirectionalTradingBacktesting())
    config_generator.trading_pair = trading_pair
    config_generator.candles_trading_pair = trading_pair

    candles = await optimizer._db_client.get_candles(CONNECTOR_NAME, trading_pair,
                                                     resolution, start_date.timestamp(), end_date.timestamp())
    start_time = candles.data["timestamp"].min()
    end_time = candles.data["timestamp"].max()
    config_generator.backtester.backtesting_data_provider.candles_feeds[
        f"{CONNECTOR_NAME}_{trading_pair}_{resolution}"] = candles.data
    config_generator.start = start_time
    config_generator.end = end_time
    await optimizer.optimize(
        study_name="test_final_mt_fixed_pct_1week_oct",
        config_generator=config_generator,
        n_trials=2,
    )

[I 2024-11-07 22:41:15,248] A new study created in RDB with name: test_final_mt_fixed_pct_1week_oct
/home/sapel/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
2024-11-07 22:42:22,665 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7faf767bea40>
2024-11-07 22:42:22,666 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7faf7681a380>, 702.46494195)]']
connector: <aiohttp.connector.TCPConnector object at 0x7faf767bfac0>
2024-11-07 22:42:29,714 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7faf767bfe50>
2024-11-07 22:42:29,716 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7faf767

In [ ]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()